<a href="https://colab.research.google.com/github/mish841/RAISE-26-Data-Science-AI-Competition-Submission/blob/mishal-phase2-progress/Data_Science_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load dataset

In [20]:
import pandas as pd
import nltk
from collections import Counter

DATA_PATH = "dataset_A_news_full_10500.csv"
# df = pd.read_csv("dataset_A_news_full_10500.csv")


df = pd.read_csv(DATA_PATH)

df.head()
df.info()
df.isna().sum().sort_values(ascending=False).head(20)
# After loading the data
df['date'] = pd.to_datetime(df['date'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   title                       10500 non-null  object
 1   link                        10500 non-null  object
 2   date                        10500 non-null  object
 3   source                      10500 non-null  object
 4   number_of_characters_title  10500 non-null  int64 
 5   number_of_words_title       10500 non-null  int64 
 6   day_of_week                 10500 non-null  object
 7   month                       10500 non-null  object
 8   year                        10500 non-null  int64 
 9   quarter                     10500 non-null  int64 
 10  is_weekend                  10500 non-null  bool  
 11  classes_str                 10500 non-null  object
dtypes: bool(1), int64(4), object(7)
memory usage: 912.7+ KB


Class Frequency

In [21]:
all_classes = []
for classes_str in df['classes_str']:
    classes = [c.strip() for c in str(classes_str).split(';')]
    all_classes.extend(classes)

# Count occurrences
class_counts = Counter(all_classes)

# Convert to DataFrame
class_df = pd.DataFrame(
    class_counts.items(),
    columns=['class', 'count']
).sort_values('count', ascending=False).reset_index(drop=True)

print(f"\nTotal unique classes found: {len(class_df)}")
print(f"\nTop 12 Most Frequent Classes:\n")
print(class_df.head(15).to_string(index=False))


Total unique classes found: 12

Top 12 Most Frequent Classes:

                                   class  count
                    Work, Jobs & Economy   2526
         Learning, Knowledge & Education   1946
                Technology & Interaction   1733
               Society, Ethics & Culture   1584
           Routine, Lifestyle & Behavior   1479
Sentiment (Positive / Negative Feelings)   1441
                             Human Roles   1256
                   Health, Safety & Risk   1230
       Creativity, Expression & Identity    877
             Cognitive & Decision-Making    832
      Social Interaction & Relationships    730
        Emotion, Motivation & Well-being    551


Domain Parsing

In [22]:
# DOMAIN_MAP = {
#     "Work": "Work, Jobs & Economy",
#     "Health": "Health, Safety & Risk",
#     "Education": "Learning, Knowledge & Education",
#     "Sentiment": "Sentiment (Positive / Negative Feelings)",
#     "Emotion": "Emotion, Motivation & Well-being",
#     "Human Roles": "Human Roles",
#     "Lifestyle": "Routine, Lifestyle & Behavior:",
#     "Cognitive": "Cognitive & Decision-Making",
#     "Creativity": "Creativity, Expression & Identity",
#     "Relationship": "Social Interaction & Relationships",
#     "Society": "Society, Ethics & Culture",
#     "Technology": "Technology & Interaction"

# }


Domain article frequency

In [27]:
# Create boolean columns for ALL 12 categories using EXACT names
df['is_sentiment'] = df['classes_str'].apply(lambda x: 'Sentiment (Positive / Negative Feelings)' in x)
df['is_emotion'] = df['classes_str'].apply(lambda x: 'Emotion, Motivation & Well-being' in x)
df['is_human_roles'] = df['classes_str'].apply(lambda x: 'Human Roles' in x)
df['is_behavior'] = df['classes_str'].apply(lambda x: 'Routine, Lifestyle & Behavior' in x)
df['is_cognitive'] = df['classes_str'].apply(lambda x: 'Cognitive & Decision-Making' in x)
df['is_creativity'] = df['classes_str'].apply(lambda x: 'Creativity, Expression & Identity' in x)
df['is_relationships'] = df['classes_str'].apply(lambda x: 'Social Interaction & Relationships' in x)
df['is_work'] = df['classes_str'].apply(lambda x: 'Work, Jobs & Economy' in x)
df['is_education'] = df['classes_str'].apply(lambda x: 'Learning, Knowledge & Education' in x)
df['is_health'] = df['classes_str'].apply(lambda x: 'Health, Safety & Risk' in x)
df['is_society'] = df['classes_str'].apply(lambda x: 'Society, Ethics & Culture' in x)
df['is_technology'] = df['classes_str'].apply(lambda x: 'Technology & Interaction' in x)



In [28]:
df.head()

,title,link,date,source,number_of_characters_title,number_of_words_title,day_of_week,month,year,quarter,...,is_human_roles,is_behavior,is_cognitive,is_creativity,is_relationships,is_work,is_education,is_health,is_society,is_technology
0,Google’s AI is the ‘worst’ for stealing conten...,https://news.google.com/rss/articles/CBMipgFBV...,2025-09-11,Fortune,74,13,Thursday,September,2025,3,...,True,False,False,False,False,False,False,False,True,False
1,Powering the Next Wave of Enterprise Innovatio...,https://news.google.com/rss/articles/CBMitgFBV...,2025-09-11,Silicon Canals,106,16,Thursday,September,2025,3,...,False,False,False,True,False,True,False,False,False,False
2,AI a ‘strategic necessity’ law lecturer says,https://news.google.com/rss/articles/CBMiiAFBV...,2025-09-11,qlsproctor.com.au,64,9,Thursday,September,2025,3,...,False,False,False,False,False,False,False,False,True,False
3,Datacom sees AI agents as pivotal to legacy ap...,https://news.google.com/rss/articles/CBMirAFBV...,2025-09-11,ARNnet,70,12,Thursday,September,2025,3,...,False,True,False,False,False,False,False,False,False,False
4,"Student Blog: Startups, AI, and Lessons from S...",https://news.google.com/rss/articles/CBMijwFBV...,2025-09-11,The University of Queensland,85,13,Thursday,September,2025,3,...,False,False,False,False,False,False,True,False,False,False


In [30]:

domain_map = {
    'Sentiment (Positive / Negative Feelings)': 'is_sentiment',
    'Emotion, Motivation & Well-being': 'is_emotion',
    'Human Roles': 'is_human_roles',
    'Routine, Lifestyle & Behavior': 'is_behavior',
    'Cognitive & Decision-Making': 'is_cognitive',
    'Creativity, Expression & Identity': 'is_creativity',
    'Social Interaction & Relationships': 'is_relationships',
    'Work, Jobs & Economy': 'is_work',
    'Learning, Knowledge & Education': 'is_education',
    'Health, Safety & Risk': 'is_health',
    'Society, Ethics & Culture': 'is_society',
    'Technology & Interaction': 'is_technology'
}

print("\nKey Research Domain Counts:")

for label, col in domain_map.items():
    count = df[col].sum()
    percentage = (count / len(df)) * 100
    print(f"{label:40} {count:5} articles ({percentage:5.1f}%)")



Key Research Domain Counts:
Sentiment (Positive / Negative Feelings)  1441 articles ( 13.7%)
Emotion, Motivation & Well-being           551 articles (  5.2%)
Human Roles                               1256 articles ( 12.0%)
Routine, Lifestyle & Behavior             1479 articles ( 14.1%)
Cognitive & Decision-Making                832 articles (  7.9%)
Creativity, Expression & Identity          877 articles (  8.4%)
Social Interaction & Relationships         730 articles (  7.0%)
Work, Jobs & Economy                      2526 articles ( 24.1%)
Learning, Knowledge & Education           1946 articles ( 18.5%)
Health, Safety & Risk                     1230 articles ( 11.7%)
Society, Ethics & Culture                 1584 articles ( 15.1%)
Technology & Interaction                  1733 articles ( 16.5%)


Sample headlines of domains

In [ ]:
# Sample headlines by domain for validation
print("\n📰 Sample Headlines by Key Domains:")
print("="*70)

for domain in ['Work', 'Health', 'Education']:
    filter_col = f'is_{domain.lower()}'
    domain_articles = df[df[filter_col]]

    print(f"\n{domain.upper()} ({len(domain_articles)} articles):")
    for idx, row in domain_articles.head(3).iterrows():
        print(f"  • {row['title']}")

Behavioral vs Emotional framing

In [ ]:
BEHAVIOR_CLASSES = {
    "Cognitive & Decision-Making",
    "Routine, Lifestyle & Behavior",
    "Work, Jobs & Economy",
    "Learning, Knowledge & Education",
    "Technology & Interaction",
    "Human Roles",
    "Society, Ethics & Culture",
    "Health, Safety & Risk"
}

EMOTION_CLASSES = {
    "Sentiment (Positive / Negative Feelings)",
    "Emotion, Motivation & Well-being",
    "Creativity, Expression & Identity",
    "Social Interaction & Relationships",
    "Health, Safety & Risk"
}


Flag each headline

In [ ]:
def has_any(classes_str, target_set):
    return any(c in target_set for c in classes_str)

df["behavior_framing"] = df["classes_str"].apply(lambda x: has_any(x, BEHAVIOR_CLASSES))
df["emotion_framing"]  = df["classes_str"].apply(lambda x: has_any(x, EMOTION_CLASSES))


Proportions of headlines describing emotional effects of AI vs behavioral effects of AI

In [ ]:
df[["behavior_framing", "emotion_framing"]].mean()
